In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/data.csv").drop_duplicates()
df = data.loc[lambda x: x["37.7745985956747"] > 40]
data = pd.concat([data, df, df]).drop_duplicates(keep=False)

crimes = {"counter":0}

def numerical(crime):
    if crime not in crimes.keys():
        crimes[crime] = crimes['counter']
        crimes['counter'] += 1
    return crimes[crime]

data['WARRANTS'] = data.apply(lambda row: numerical(row['WARRANTS']), axis=1)

def bucketer(mini, maxi, buckets, number):
    if(number == maxi):
        return buckets - 1
    bucket_size = (maxi - mini)/buckets
    num_norm = number - mini
    return num_norm//bucket_size

lat_max = max(data["37.7745985956747"])
lat_min = min(data["37.7745985956747"])

data['lat'] = data.apply(lambda row: bucketer(lat_min, lat_max, 25, row["37.7745985956747"]), axis=1)

long_max = max(data["-122.425891675136"])
long_min = min(data["-122.425891675136"])

data['long'] = data.apply(lambda row: bucketer(long_min, long_max, 25, row["-122.425891675136"]), axis=1)
data["cell"] = data["lat"]*25 + data["long"]

num_crimes = crimes.pop("counter")
data.head()
        

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB

y = data.drop(["-122.425891675136", "WARRANTS", "05", "37.7745985956747", "lat", "cell"], axis = 1)
X = data.drop(["-122.425891675136", "37.7745985956747", "long", "lat", "cell"], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

long_predictor = CategoricalNB().fit(X_train, y_train)
long_predictor.score(X_test, y_test)


In [ ]:
y = data.drop(["-122.425891675136", "WARRANTS", "05", "37.7745985956747", "long", "cell"], axis = 1)
X = data.drop(["-122.425891675136", "37.7745985956747", "long", "lat", "cell"], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

lat_predictor = CategoricalNB().fit(X_train, y_train)
lat_predictor.score(X_test, y_test)

In [ ]:
def normalize(number, mini, maxi, buckets):
    bucket_size = (maxi - mini)/buckets
    return number*bucket_size + mini
def predict(month, crime, lat, long):
    longs = long.predict_proba([[month, crime]]).reshape((25))
    lats = lat.predict_proba([[month, crime]]).reshape((25)).reshape((-1,1))
    grid = np.multiply(longs, lats)
    grid = (5/np.max(grid))*grid
    dic = {}
    for i in range(25):
        for j in range(25):
            dic[(i,j)] = grid[i][j]
    l = sorted(dic.items(), key=lambda item: item[1])[-50:]
    print(l[0])
    
    li = [{"Latitude": normalize(i[0][1], lat_min, lat_max, 25), "Longitude": normalize(i[0][0], long_min, long_max, 25), "Weight": i[1]} for i in l]
    #li = [{"Latitude": i[0][0], "Longitude": i[0][1], "Weight": i[1]} for i in l]
    return li
    
    
    
json = { i: {crime: predict(i, crimes[crime], lat_predictor, long_predictor) for crime in crimes.keys()} for i in range(12)}

In [ ]:
json[1]

In [ ]:
final_json = json

In [ ]:
import json
with open('result.json', 'w') as fp:
    json.dump(final_json, fp, indent = 4, sort_keys = False)